In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(context='paper', style='whitegrid', rc={'figure.figsize':(21/2, 9/2), 'figure.dpi': 120})
color_palette = sns.color_palette("Set2")
sns.set_palette(color_palette)

seed_ = 20200218
np.random.seed(seed_)

import os
from PIL import Image

import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.callbacks import EarlyStopping

In [2]:
# tensorflow use gpu https://www.tensorflow.org/guide/gpu

In [3]:
config = tf.config.experimental.list_physical_devices('GPU')

In [4]:
train_dataset_path = "./data/train/"
val_dataset_path = "./data/val/"
test_dataset_path = "./data/test/"

In [5]:
train_data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=(1.0/255)
)

val_data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=(1.0/255)
)

test_data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=(1.0/255)
)


In [6]:
train_generator = train_data_generator.flow_from_directory(
    directory=train_dataset_path,
    class_mode='binary',
    target_size=(224, 224),
    color_mode="grayscale",
    batch_size=128,
)

val_generator = val_data_generator.flow_from_directory(
    directory=val_dataset_path,
    class_mode='binary',
    target_size=(224, 224),
    color_mode="grayscale",
)

test_generator = test_data_generator.flow_from_directory(
    directory=test_dataset_path,
    class_mode='binary',
    target_size=(224, 224),
    color_mode="grayscale",
)

Found 5076 images belonging to 2 classes.
Found 156 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Model

In [7]:
model = Sequential()

model.add(Conv2D(8, kernel_size=(5, 5), input_shape=(224, 224, 1), padding="same", activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# FC
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam' , loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 8)       208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 16)      1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 48400)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6195328   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [8]:
history = model.fit_generator(train_generator, epochs=8, validation_data=test_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/8
40/40 [==============================] - 28s 692ms/step - loss: 0.9690 - accuracy: 0.7043 - val_loss: 0.5924 - val_accuracy: 0.6314
Epoch 2/8
40/40 [==============================] - 28s 693ms/step - loss: 0.2177 - accuracy: 0.9133 - val_loss: 0.9447 - val_accuracy: 0.7019
Epoch 3/8
40/40 [==============================] - 28s 689ms/step - loss: 0.0924 - accuracy: 0.9665 - val_loss: 0.7519 - val_accuracy: 0.7853
Epoch 4/8
40/40 [==============================] - 28s 694ms/step - loss: 0.0693 - accuracy: 0.9736 - val_loss: 0.8050 - val_accuracy: 0.7788
Epoch 5/8
40/40 [==============================] - 27s 684ms/step - loss: 0.0541 - accuracy: 0.9813 - val_loss: 1.1408 - val_accuracy: 0.7292
Epoch 6/8
40/40 [==============================] - 28s 708ms/step - loss: 0.0502 - accuracy: 0.9817 - val_loss: 0.8275 - val_accuracy: 0.7853
Epoch 7/8
40/40 [==============================] - 27s 687ms/step - loss

In [ ]:
model.evaluate_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [ ]:
epoch_x = np.arange(1, len(history.history['accuracy'])+1)

plt.figure(figsize=(8, 4), dpi=100)
sns.lineplot(epoch_x, history.history['accuracy'], label='train')
sns.lineplot(epoch_x, history.history['val_accuracy'], label='validation')
plt.title('accuracy')
plt.xticks(epoch_x)
plt.show()

plt.figure(figsize=(8, 4), dpi=100)
sns.lineplot(epoch_x, history.history['loss'], label='train')
sns.lineplot(epoch_x, history.history['val_loss'], label='validation')
plt.title('loss')
plt.xticks(epoch_x)
plt.show()